In [3]:
import torch
import yaml
import numpy as np
import pickle
from src.datasets import RetrievalDatasetWithoutEmb
from src.models import LLMs
from src.utils import print_log

/home/comp/cscxliu/miniconda3/envs/derek/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/comp/cscxliu/miniconda3/envs/derek/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/comp/cscxliu/miniconda3/envs/derek/lib/python3.10/site-packages/torch_geometric/typing.py:110: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /home/comp/cscxliu/miniconda3/envs/derek/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/home/comp/cscxliu/miniconda3/envs/derek/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedo

In [2]:
def print_(v):
    for i in v:
        print(i)

In [3]:
def get_all_triplets(sample):
    all_entities = sample["text_entity_list"] + sample["non_text_entity_list"]
    all_relations = sample["relation_list"]
    h_id_list, r_id_list, t_id_list = sample["h_id_list"], sample["r_id_list"], sample["t_id_list"]
    all_triplets = [(all_entities[h], all_relations[r], all_entities[t]) for (h,r,t) in zip(h_id_list, r_id_list, t_id_list)]
    return all_triplets

In [4]:
import torch
llm_refined_data = torch.load("logging_detection/grailqa/gpt-4o-mini/DDE/path-detection-test/evaluation.pth")

/tmp/ipykernel_3164316/3524735000.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  llm_refined_data = torch.load("logging_detection/grailqa/gpt-4o-mini/DDE/path-detection

In [5]:
org_data = torch.load("datasets/grailqa/processed/intermediate.pth")

/tmp/ipykernel_3164316/1227355272.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  org_data = torch.load("datasets/grailqa/processed/intermediate.pth")


In [6]:
dataset = pickle.load(open("datasets/grailqa/processed/train.pkl", "rb"))

In [7]:
org_data.update(llm_refined_data)

In [8]:
shortest_paths_cache = torch.load("datasets/grailqa/processed/shortest_path.pth")

/tmp/ipykernel_3164316/612739741.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  shortest_paths_cache = torch.load("datasets/grailqa/processed/shortest_path.pth")


In [9]:
def get_centric_grpah(all_triplets, q):
    selections = []
    for triple in all_triplets:
        if triple[0] == q and not triple[-1].startswith(('m.', 'g.')):
            selections.append([triple])
        if triple[-1] == q and not triple[0].startswith(('m.', 'g.')):
            selections.append([triple])
    return selections

In [10]:
def get_all_triplets(sample):
    all_entities = sample["text_entity_list"] + sample["non_text_entity_list"]
    all_relations = sample["relation_list"]
    h_id_list, r_id_list, t_id_list = sample["h_id_list"], sample["r_id_list"], sample["t_id_list"]
    all_triplets = [(all_entities[h], all_relations[r], all_entities[t]) for (h, r, t) in zip(h_id_list, r_id_list, t_id_list)]
    return all_triplets

def get_logic_from_path(path):
    return tuple(triple[1] for triple in path)

def get_entities_from_path(path):
    return [triple[0] for triple in path] + [path[-1][-1]]

In [15]:
org_new = {}
for id, selection_paths in org_data.items():
    sample = [d for d in dataset if d['id'] == id][0]
    q_entities = sample['q_entity']
    if not selection_paths or max(len(path) for path in selection_paths) == 0 or sample["a_entity"] == ['null']:
        org_new[id] = []
    # get template logic, we aim to find candidates whose logic is in templates
    template_logic_q = [get_logic_from_path(p) for p in selection_paths if p[0][0] in q_entities or p[-1][-1] in q_entities]
    template_logic_a = [get_logic_from_path(p) for p in selection_paths if not (p[0][0] in q_entities or p[-1][-1] in q_entities)]

    candidates_q, candidates_a, final_select = [], [], []
    shortest_paths = shortest_paths_cache[id]
    # For normal cases, candidates include shortest paths
    candidates_q += shortest_paths
    
    all_triplets = get_all_triplets(sample)
    if sample['function'] == 'count':
        # For count type, candidate include Q-centric grpah
        candidates_q = get_centric_grpah(all_triplets, sample['q_entity'][0])
    if sample['function'] not in ['count', 'none']:
        # For others, candidate include A-centric graph
        seed_set = set()
        for p in shortest_paths:
            seed_set |= set(get_entities_from_path(p))
        seed_set -= set(sample['q_entity'])
        
        for a in seed_set:
            candidates_a += get_centric_grpah(all_triplets, a)
    
    for p in candidates_q:
        if get_logic_from_path(p) in template_logic_q:
            final_select.append(p)
    for p in candidates_a:
        if get_logic_from_path(p) in template_logic_a:
            final_select.append(p)
    org_new[id] = final_select


In [16]:
torch.save(org_new, "datasets/grailqa/processed/refined_path.pth")